In [131]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import nltk
import string
from nltk import word_tokenize
from nltk.corpus import stopwords
import string
from nltk.stem import PorterStemmer

## 1. Open the Initial EDA 

In [132]:
data = pd.read_csv("initial_eda.csv")

In [133]:
data.head(5)

,submission_id,submission_score,submission_title,submission_link_flair_text,submission_selftext,reply_author,reply_body,reply_created_utc,word_count
0,aglcrj,1,Goodnotes 4 vs. Goodnotes 5 right now,NaN,I have used Goodnotes 4 for work a ton. And I do mean a ton. I bought and downloaded 5 yesterd...,Mauri97,"I'm getting a ton of bugs with 5 as well (snappy lines, no response at times) and for some reaso...",1547656134,206
1,aglcrj,1,Goodnotes 4 vs. Goodnotes 5 right now,NaN,I have used Goodnotes 4 for work a ton. And I do mean a ton. I bought and downloaded 5 yesterd...,nathanwj,Goodnotes 5 is not yet compatible with the desktop app as it says in the release notes. There wi...,1547658625,115
2,aglcrj,1,Goodnotes 4 vs. Goodnotes 5 right now,NaN,I have used Goodnotes 4 for work a ton. And I do mean a ton. I bought and downloaded 5 yesterd...,Rowyfo,I haven't downloaded GN5 yet but watched a walkthrough and I did see that there's a snap option ...,1547662874,138
3,aglcrj,1,Goodnotes 4 vs. Goodnotes 5 right now,NaN,I have used Goodnotes 4 for work a ton. And I do mean a ton. I bought and downloaded 5 yesterd...,Mauri97,"That sounds a lot like what's going on, thanks!",1547663080,39
4,aglcrj,1,Goodnotes 4 vs. Goodnotes 5 right now,NaN,I have used Goodnotes 4 for work a ton. And I do mean a ton. I bought and downloaded 5 yesterd...,nongaussian,I will probably stick to GoodNotes 4 for a while. Noticed that in GoodNotes 5 there is no easy w...,1547675267,239


## 2. Removing hyperlinks and HEX code

In [134]:
# Copying raw reply_body for future troubleshooting
data['raw_reply_body'] = data['reply_body']

In [135]:
URL_PATTERN = r'[A-Za-z0-9]+://[A-Za-z0-9%-_]+(/[A-Za-z0-9%-_])*(#|\\?)[A-Za-z0-9%-_&=]*'
HEX_PATTERN = r'[^\x00-\x7f]'

data['reply_body'] = data['reply_body'].str.replace(URL_PATTERN,'')
data['reply_body'] = data['reply_body'].str.replace(HEX_PATTERN,'')

/var/folders/1x/hvb1q6g96_qgcywwm07fmxhc0000gn/T/ipykernel_8392/2375308378.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  data['reply_body'] = data['reply_body'].str.replace(URL_PATTERN,'')
/var/folders/1x/hvb1q6g96_qgcywwm07fmxhc0000gn/T/ipykernel_8392/2375308378.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  data['reply_body'] = data['reply_body'].str.replace(HEX_PATTERN,'')


## 3/4/5/6. Preprocessing
* Remove punctuations
* Convert to lowercase
* Remove StopWords
* Stem the words

In [136]:
def process(text):
    stoplist = set(stopwords.words('english'))
    ptStemmer = PorterStemmer()
    
    word_list = []
    
    text_lower = text.lower()
    
    # Getting rid of punctuation (3)
    text_lower = "".join([char for char in text_lower if char not in string.punctuation])
    
    # To Lower conversion is happening here (4)
    for word in word_tokenize(text_lower):
        #print(word)
        # Checking for stopWords (5)
        if word in stoplist:
            #print("Stripping stopword:",word)
            continue;
            
        # Removing words that are less than 3 characters long
        if len(word) <3:
            continue;
    
        # Stemming the word (6)
        word_list.append(ptStemmer.stem(word))
        
    # Combining list back to single string
    return ' '.join(word_list)

In [137]:
data['reply_body'] = data['reply_body'].apply(process)

In [142]:
# Removing empty reply_bodies (After preprocessing)
data = data[data['reply_body'].str.len()>1]

In [143]:
pd.set_option('display.max_colwidth', 100)
data['reply_body'][0:5]

0    get ton bug well snappi line respons time reason abil sync googl drive download multipl file goo...
1                                    goodnot yet compat desktop app say releas note featur ad near futur
2    havent download gn5 yet watch walkthrough see there snap option check uncheck think somewher pen...
3                                                                           sound lot like what go thank
4    probabl stick goodnot notic goodnot easi way export note dropbox pdf format love avail nonappl d...
Name: reply_body, dtype: object

## 7. Export the preprocessed DataFrame

In [144]:
data.to_csv("preprocessed.csv", index=False)